In [1]:
using SapoDatasets
using Flux

In [2]:
il = ImageList_fromfolder("Images", 500)

ImageList(["Images/Cats/cat1.jpeg", "Images/Cats/cat2.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], SapoDatasets.imagelistget, 500)

In [3]:
sil = SplitImageList_byrandpct(il)

SplitImageList(["Images/Cats/cat1.jpeg", "Images/Cats/cat2.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], SapoDatasets.imagelistget, 500, [1, 3, 4, 5, 6], [2])

In [4]:
lil = LabeledImageList_fromfolder(sil)

LabeledImageList(["Images/Cats/cat1.jpeg", "Images/Cats/cat2.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], Any["Cats", "Cats", "Cats", "Dogs", "Dogs", "Dogs"], SapoDatasets.imagelistget, identity, 500, [1, 3, 4, 5, 6], [2])

In [5]:
train_dset = getimage_dset(lil)

ImageDataset(["Images/Cats/cat1.jpeg", "Images/Cats/cat3.jpeg", "Images/Dogs/dog1.jpeg", "Images/Dogs/dog2.jpeg", "Images/Dogs/dog3.jpeg"], Any["Cats", "Cats", "Dogs", "Dogs", "Dogs"], SapoDatasets.imagelistget, identity, 500, "train")

In [6]:
valid_dset = getimage_dset(lil, "valid")

ImageDataset(["Images/Cats/cat2.jpeg"], Any["Cats"], SapoDatasets.imagelistget, identity, 500, "valid")

In [7]:
flatten(x) = reshape(x, 64, 1)

flatten (generic function with 1 method)

In [8]:
expand(x) = reshape(x, size(x)..., 1)

expand (generic function with 1 method)

In [9]:
model = Chain(expand, Conv((3,3), 3=>16, pad=1),Conv((3,3), 16=>32, pad=1,stride=2), Conv((3,3), 32=>64, pad=1, stride=2),
            MeanPool((125,125)), flatten, Dense(64, 2), softmax)

Chain(expand, Conv((3, 3), 3=>16), Conv((3, 3), 16=>32), Conv((3, 3), 32=>64), MeanPool((125, 125), pad = (0, 0, 0, 0), stride = (125, 125)), flatten, Dense(64, 2), softmax)

In [10]:
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [11]:
function loss(x, y)
    ŷ = model(x)[1]
    y = y == "Cat" ? 1 : 0
    Flux.crossentropy([ŷ], [y])
end

loss (generic function with 1 method)

In [12]:
ps = params(model)

Params([Float32[-0.4266919 -0.2868885 0.30015346; 0.025407204 0.36599183 -0.24867643; 0.0894827 0.40508714 0.122271486]

Float32[-0.28557205 0.033910885 0.29044345; 0.20829326 0.31660777 0.18946218; -0.45364296 -0.1507342 -0.16836925]

Float32[0.42470068 -0.04856856 0.09254394; -0.37976474 0.34444454 0.2671528; -0.0797525 0.2377322 -0.41468737]

Float32[-0.13968472 -0.2801205 -0.028975118; 0.35815063 -0.27400503 0.23190384; -0.20115569 0.034996547 -0.47966653]

Float32[-0.2563006 -0.017318984 -0.08139332; 0.3717099 -0.40295938 -0.303682; -0.38172126 0.01659412 0.080233134]

Float32[0.46160612 0.40428016 -0.19944666; 0.3414494 -0.34298933 0.39589468; 0.44736505 0.20220993 -0.06459398]

Float32[-0.24438483 0.3829152 -0.46633103; -0.31547186 0.031832766 -0.3019419; -0.19671784 0.42537963 0.24226278]

Float32[-0.35786366 0.22435687 0.010598385; 0.04905492 0.3947754 -0.18687119; -0.20834057 -0.26001218 0.47583747]

Float32[-0.022218777 -0.17595148 -0.4666831; -0.3408979 0.3186228 -0.2012712

In [13]:
Flux.train!(loss, ps, train_dset, opt)

In [14]:
x = valid_dset[1]

(FixedPointNumbers.Normed{UInt8,8}[0.827N0f8 0.769N0f8 … 0.914N0f8 0.247N0f8; 0.831N0f8 0.749N0f8 … 0.906N0f8 0.4N0f8; … ; 0.761N0f8 0.635N0f8 … 0.396N0f8 0.733N0f8; 0.761N0f8 0.749N0f8 … 0.298N0f8 0.863N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.792N0f8 0.91N0f8 … 0.592N0f8 0.851N0f8; 0.737N0f8 0.902N0f8 … 0.494N0f8 0.827N0f8; … ; 0.584N0f8 0.408N0f8 … 0.161N0f8 0.447N0f8; 0.537N0f8 0.31N0f8 … 0.153N0f8 0.408N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.373N0f8 0.6N0f8 … 0.78N0f8 0.71N0f8; 0.424N0f8 0.549N0f8 … 0.776N0f8 0.694N0f8; … ; 0.549N0f8 0.176N0f8 … 0.694N0f8 0.678N0f8; 0.627N0f8 0.169N0f8 … 0.729N0f8 0.667N0f8], "Cats")

In [15]:
model(x[1])

Tracked 2×1 Array{Float32,2}:
 0.9971089f0   
 0.0028911808f0